In [1]:
from IPython.display import display
import time
import pandas as pd
import os
import shutil
from sqlalchemy import *
from ratio_sql import ratio_calculation
from urllib.parse import quote_plus
from ic_sql import income_statement
from balancesheet_sql import balance_sheet
from cf_sql import cash_flow
from price_history_sql import price_history




In [2]:
# Database connection setup
db_user = 'tkan'
db_password = 'Maihainganha@1'
db_host = 'localhost'  
db_port = '5432'       
db_name = 'finance_db' 
password = quote_plus(db_password) # Encode password
connection_str = f'postgresql://{db_user}:{password}@{db_host}:{db_port}/{db_name}' # Connection string
engine = create_engine(connection_str) # Create engine
inspector = inspect(engine) # Create inspector
df_tickers = pd.read_sql('SELECT "Ticker" FROM analysis_data.companies_list', engine)
ticker_list = set(df_tickers['Ticker'].str.strip())

In [ ]:

# truyền dữ liệu ratio (đang lỗi chốc xử lý)
failed_tickers = []
for ticker in ticker_list: 
    try: 
        df_ratio = ratio_calculation(ticker)
        df_ratio = df_ratio.set_index('Chỉ số')
        df_ratio.loc['Ticker'] = ticker
        df_ratio = df_ratio.T
        df_ratio.index.name = 'year'
        df_ratio.columns.name = None
        df_ratio = df_ratio.reset_index()
        if inspector.has_table('financial_ratio'):
            with engine.connect() as conn:
                conn.execute(text(f"DELETE FROM raw.financial_ratio WHERE \"Ticker\" = '{ticker}'"))
                conn.commit()
        df_ratio.to_sql('financial_ratio', engine ,schema = 'raw', if_exists='append', index=False)
    except Exception as e:
        print(f"Lỗi với ticker {ticker}: {e}")
        failed_tickers.append({'ticker': ticker, 'error': str(e)})
    time.sleep(3)  # Thêm độ trễ 5 giây giữa các lần lặp

if os.path.exists('data_processed'):
    shutil.rmtree('data_processed')

if os.path.exists('data_raw'):
    shutil.rmtree('data_raw')
    




In [ ]:
# truyền dữ liệu cash flow
indirect_tickers = set()
direct_tickers = set()
if inspector.has_table('cash_flow_indirect', schema='raw'):
    df_all_indirect = pd.read_sql('SELECT "Ticker" FROM raw.cash_flow_indirect', engine)
    indirect_tickers = set(df_all_indirect['Ticker'].str.strip())
if inspector.has_table('cash_flow_direct', schema='raw'):
    df_all_direct = pd.read_sql('SELECT "Ticker" FROM raw.cash_flow_direct', engine)
    direct_tickers = set(df_all_direct['Ticker'].str.strip())
missing_tickers = list(ticker_list - (indirect_tickers | direct_tickers))
tickers_failed_cash_flow = []
print(missing_tickers)

for i, ticker in enumerate(missing_tickers):
    try:
        target_table = 'unknown'
        df_cf = cash_flow(ticker)
        df_cf = df_cf.set_index(df_cf.columns[0])
        df_cf.loc['Ticker'] = ticker
        df_cf = df_cf.T
        df_cf.columns.name = None
        df_cf.index.name = 'year'
        df_cf = df_cf.reset_index()
        df_cf.columns = df_cf.columns.str.strip()
        if '1. Tiền thu từ bán hàng, cung cấp dịch vụ và doanh thu khác' in df_cf.columns:
            target_table = 'cash_flow_direct'
        elif '2. Điều chỉnh cho các khoản' in df_cf.columns:
            target_table = 'cash_flow_indirect'
        if inspector.has_table(target_table, schema='raw'):
            with engine.connect() as conn:
                conn.execute(text(f"DELETE FROM raw.{target_table} WHERE \"Ticker\" = '{ticker}'"))
                conn.commit()
        df_cf.to_sql(target_table, engine, schema='raw', if_exists='append', index=False)
        print(f"Đã xử lý xong ticker {i+1}/{len(missing_tickers)}: {ticker} vào bảng {target_table}")
    except Exception as e:
        print(f"Lỗi với ticker {ticker}: {e}")
        tickers_failed_cash_flow.append({'ticker': ticker, 'error': str(e)})
    time.sleep(1)  # Thêm độ trễ 5 giây giữa các lần lặp
    
print(tickers_failed_cash_flow)
    
    

[]
[]


In [5]:
#truyền dữ liệu income statement
income_tickers = set()
if inspector.has_table('income_statement', schema='raw'):
    df_all_income = pd.read_sql('SELECT "Ticker" FROM raw.income_statement', engine)
    income_tickers = set(df_all_income['Ticker'].str.strip())
missing_tickers = set(ticker_list - income_tickers)
print(missing_tickers)
    
failed_tickers_income =[]
for i, ticker in enumerate(missing_tickers):
    try: 
        df_ic = income_statement(ticker)
        df_ic = df_ic.set_index(ticker)
        df_ic.loc['Ticker'] = ticker 
        df_ic = df_ic.T
        df_ic.columns.name = None
        df_ic.index.name = 'year'
        df_ic = df_ic.reset_index()

        if inspector.has_table('income_statement'):
            with engine.connect() as conn:
                conn.execute(text(f"DELETE FROM raw.income_statement WHERE \"Ticker\" = '{ticker}'"))
                conn.commit()
        df_ic.to_sql('income_statement', engine, schema='raw', if_exists='append', index=False)
        print(f"Đã xử lý xong ticker {i+1}/{len(missing_tickers)}: {ticker}")

    except Exception as e:
        print(f"Lỗi với ticker {ticker}: {e}")
        failed_tickers_income.append({'ticker': ticker, 'error': str(e)})
    time.sleep(3)

print(failed_tickers_income)


set()
[]


In [ ]:
#truyền dữ liệu balance sheet
failed_tickers_balance =[]
for i, ticker in enumerate(ticker_list):
    try: 
        df_balancesheet = balance_sheet(ticker)
        df_balancesheet = df_balancesheet.set_index(df_balancesheet.columns[0])
        df_balancesheet.loc['Ticker'] = ticker
        df_balancesheet = df_balancesheet.T
        df_balancesheet.columns.name = None
        df_balancesheet.index.name = 'year'
        df_balancesheet = df_balancesheet.reset_index()
        if inspector.has_table('balance_sheet'):
            with engine.connect() as conn:
                conn.execute(text(f"DELETE FROM raw.balance_sheet WHERE \"Ticker\" = '{ticker}'"))
                conn.commit()
        df_balancesheet.to_sql('balance_sheet', engine, schema='raw', if_exists='append', index=False)
        print(f"Đã xử lý xong ticker {i+1}/{len(ticker_list)}: {ticker}")
    except Exception as e:
        print(f"Lỗi với ticker {ticker}: {e}")
    time.sleep(3)  # Thêm độ trễ 5 giây giữa các lần lặp

pd.DataFrame(failed_tickers_balance).to_excel('failed_tickers_balance.xlsx', engine='openpyxl')

In [ ]:
#bài 1: lọc cổ phiếu theo P/E (5 lớn nhất, 5 rẻ nhất)
# bắt đầu lọc ra 5 cổ phiếu có P/E rẻ nhất (năm mới nhất)
query_cheap_pe = """
WITH LatestData AS (
    SELECT 
        "Ticker",
        year,
        "P/E",
        ROW_NUMBER() OVER(PARTITION BY "Ticker" ORDER BY year DESC) as rn 
    FROM 
        financial_ratio
    WHERE 
        "P/E" IS NOT NULL
)
SELECT 
    "Ticker",
    year,
    "P/E"
FROM 
    LatestData
WHERE 
    rn = 1 
    AND "P/E" > 0  -- Chỉ lấy P/E "rẻ", không lấy P/E âm (lỗ)
ORDER BY 
    "P/E" ASC  -- Sắp xếp P/E tăng dần
LIMIT 5;
"""
df_pe_re_nhat = pd.read_sql_query(query_cheap_pe, engine)
display(df_pe_re_nhat)

# bắt đầu lọc ra 5 cổ phiếu có P/E lớn nhất (năm mới nhất)
query_expensive_pe = """
WITH LatestData AS (
    SELECT 
        "Ticker",
        year,
        "P/E",
        ROW_NUMBER() OVER(PARTITION BY "Ticker" ORDER BY year DESC) as rn 
    FROM
        financial_ratio
    WHERE 
        "P/E" IS NOT NULL
)
SELECT 
    "Ticker",
    year,
    "P/E"
FROM 
    LatestData
WHERE 
    rn = 1 
    AND "P/E" > 0  -- Chỉ lấy P/E đắt, không lấy P/E âm (lỗ)
ORDER BY
    "P/E" DESC
LIMIT 5;
"""
df_expensive_pe = pd.read_sql_query(query_expensive_pe, engine)
display(df_expensive_pe)

,Ticker,year,P/E
0,VRE,2024,9.75
1,HPG,2024,11.82
2,VNM,2024,13.02
3,VIC,2024,13.21
4,GAS,2024,14.44


,Ticker,year,P/E
0,MSN,2024,52.98
1,GVR,2024,30.00
2,FPT,2024,24.62
3,MWG,2024,23.62
4,PLX,2024,16.24
